# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 9 2022 3:59PM,250441,22,11102022,FUJIFILM Diosynth Biotechnologies Texas LLC,Executing
1,Nov 9 2022 3:49PM,250440,10,MTR-22-0266.1,"Senseonics, Incorporated",Released
2,Nov 9 2022 3:33PM,250439,19,8724107,Eywa Pharma Inc.,Released
3,Nov 9 2022 3:33PM,250439,19,8732662,Eywa Pharma Inc.,Released
4,Nov 9 2022 3:33PM,250439,19,8724096-BO,Eywa Pharma Inc.,Released
5,Nov 9 2022 3:04PM,250437,10,MSP215690,"Methapharm, Inc.",Released
6,Nov 9 2022 3:04PM,250437,10,MSP215694,"Methapharm, Inc.",Released
7,Nov 9 2022 3:04PM,250437,10,MSP215696,"Methapharm, Inc.",Released
8,Nov 9 2022 3:04PM,250437,10,MSP215697,"Methapharm, Inc.",Released
9,Nov 9 2022 3:04PM,250437,10,MSP215700,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,250436,Released,1
21,250437,Released,6
22,250439,Released,3
23,250440,Released,1
24,250441,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250436,NaN,NaN,1.0
250437,NaN,NaN,6.0
250439,NaN,NaN,3.0
250440,NaN,NaN,1.0
250441,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250364,8.0,3.0,3.0
250377,19.0,1.0,0.0
250384,0.0,1.0,0.0
250385,0.0,1.0,0.0
250386,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250364,8,3,3
250377,19,1,0
250384,0,1,0
250385,0,1,0
250386,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250364,8,3,3
1,250377,19,1,0
2,250384,0,1,0
3,250385,0,1,0
4,250386,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250364,8,3,3
1,250377,19,1,
2,250384,,1,
3,250385,,1,
4,250386,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 9 2022 3:59PM,250441,22,FUJIFILM Diosynth Biotechnologies Texas LLC
1,Nov 9 2022 3:49PM,250440,10,"Senseonics, Incorporated"
2,Nov 9 2022 3:33PM,250439,19,Eywa Pharma Inc.
5,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc."
11,Nov 9 2022 3:04PM,250436,10,"Methapharm, Inc."
12,Nov 9 2022 2:41PM,250434,10,ISDIN Corporation
25,Nov 9 2022 2:39PM,250431,19,ACG North America LLC
28,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC"
29,Nov 9 2022 1:42PM,250422,21,"NBTY Global, Inc."
30,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 9 2022 3:59PM,250441,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1,
1,Nov 9 2022 3:49PM,250440,10,"Senseonics, Incorporated",,,1
2,Nov 9 2022 3:33PM,250439,19,Eywa Pharma Inc.,,,3
3,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc.",,,6
4,Nov 9 2022 3:04PM,250436,10,"Methapharm, Inc.",,,1
5,Nov 9 2022 2:41PM,250434,10,ISDIN Corporation,,6,7
6,Nov 9 2022 2:39PM,250431,19,ACG North America LLC,,,3
7,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC",,,1
8,Nov 9 2022 1:42PM,250422,21,"NBTY Global, Inc.",,,1
9,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",,,83


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 3:59PM,250441,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1,
1,Nov 9 2022 3:49PM,250440,10,"Senseonics, Incorporated",1,,
2,Nov 9 2022 3:33PM,250439,19,Eywa Pharma Inc.,3,,
3,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc.",6,,
4,Nov 9 2022 3:04PM,250436,10,"Methapharm, Inc.",1,,
5,Nov 9 2022 2:41PM,250434,10,ISDIN Corporation,7,6,
6,Nov 9 2022 2:39PM,250431,19,ACG North America LLC,3,,
7,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC",1,,
8,Nov 9 2022 1:42PM,250422,21,"NBTY Global, Inc.",1,,
9,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",83,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 3:59PM,250441,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1,
1,Nov 9 2022 3:49PM,250440,10,"Senseonics, Incorporated",1,,
2,Nov 9 2022 3:33PM,250439,19,Eywa Pharma Inc.,3,,
3,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc.",6,,
4,Nov 9 2022 3:04PM,250436,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 3:59PM,250441,22,FUJIFILM Diosynth Biotechnologies Texas LLC,NaN,1.0,NaN
1,Nov 9 2022 3:49PM,250440,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Nov 9 2022 3:33PM,250439,19,Eywa Pharma Inc.,3.0,NaN,NaN
3,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc.",6.0,NaN,NaN
4,Nov 9 2022 3:04PM,250436,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 3:59PM,250441,22,FUJIFILM Diosynth Biotechnologies Texas LLC,0.0,1.0,0.0
1,Nov 9 2022 3:49PM,250440,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Nov 9 2022 3:33PM,250439,19,Eywa Pharma Inc.,3.0,0.0,0.0
3,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc.",6.0,0.0,0.0
4,Nov 9 2022 3:04PM,250436,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1752936,17.0,7.0,19.0
15,500780,86.0,3.0,8.0
19,500870,6.0,0.0,0.0
20,250424,1.0,0.0,0.0
21,2003133,6.0,2.0,0.0
22,250441,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1752936,17.0,7.0,19.0
1,15,500780,86.0,3.0,8.0
2,19,500870,6.0,0.0,0.0
3,20,250424,1.0,0.0,0.0
4,21,2003133,6.0,2.0,0.0
5,22,250441,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,17.0,7.0,19.0
1,15,86.0,3.0,8.0
2,19,6.0,0.0,0.0
3,20,1.0,0.0,0.0
4,21,6.0,2.0,0.0
5,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,17.0
1,15,Released,86.0
2,19,Released,6.0
3,20,Released,1.0
4,21,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,19.0,8.0,0.0,0.0,0.0,0.0
Executing,7.0,3.0,0.0,0.0,2.0,1.0
Released,17.0,86.0,6.0,1.0,6.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,19.0,8.0,0.0,0.0,0.0,0.0
1,Executing,7.0,3.0,0.0,0.0,2.0,1.0
2,Released,17.0,86.0,6.0,1.0,6.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,19.0,8.0,0.0,0.0,0.0,0.0
1,Executing,7.0,3.0,0.0,0.0,2.0,1.0
2,Released,17.0,86.0,6.0,1.0,6.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()